[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1GCmTVcXSJQvyXR8H0VD0lZ6aE_gm6tnf#scrollTo=ILb1FeTDcnmC)

#### Fine-tune T5 for the argument-keypoint matching pipeline
#### Input: Arguments
#### Output: Intermediary text

See the allocated GPU at first!

In [1]:
!nvidia-smi

Sat May 21 14:14:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install the simple transformers, tqdm, pandas packages

In [3]:
!pip install simpletransformers tqdm pandas

Import the train, dev and test sets

In [4]:
import pandas as pd
train_df_prev = pd.read_csv("train.csv")
dev_df_prev = pd.read_csv("dev.csv")
test_df_prev = pd.read_csv("test.csv")

In [5]:
train_df_prev.head(5)

,Unnamed: 0,topic,argument,key_point,stance,label
0,10137,We should ban human cloning,a person created through cloning could potenti...,Cloning is not understood enough yet,1,0
1,10138,We should ban human cloning,a person created through cloning could potenti...,Cloning is unethical/anti-religious,1,0
2,10139,We should ban human cloning,a person created through cloning could potenti...,Cloning is unnatural,1,0
3,10140,We should ban human cloning,a person created through cloning could potenti...,Cloning is unsafe,1,1
4,10141,We should ban human cloning,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,1,0


In [6]:
train_df_prev.columns

Index(['Unnamed: 0', 'topic', 'argument', 'key_point', 'stance', 'label'], dtype='object')

We only need the 'topic', 'argument', 'keypoint' and 'label' columns

In [7]:
train_df = train_df_prev.filter(['topic' ,'argument', 'key_point', 'label'], axis=1)
dev_df = dev_df_prev.filter(['topic', 'argument', 'key_point', 'label'], axis=1)
test_df = test_df_prev.filter(['topic', 'argument', 'key_point', 'label'], axis=1)

In [8]:
train_df.head(5)

,topic,argument,key_point,label
0,We should ban human cloning,a person created through cloning could potenti...,Cloning is not understood enough yet,0
1,We should ban human cloning,a person created through cloning could potenti...,Cloning is unethical/anti-religious,0
2,We should ban human cloning,a person created through cloning could potenti...,Cloning is unnatural,0
3,We should ban human cloning,a person created through cloning could potenti...,Cloning is unsafe,1
4,We should ban human cloning,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,0


There are two prompt engineering templates for the task.
Template 1: [X]. This means [Z].
Template 2: What are the keypoints for the following argument? [X] [Z]
where, X is the argument as input and Z is the intermediary text as output.

You need to select either template 1 or template 2 for fine-tuning BART

This is template 1

In [9]:
# Template 1
# For Train set

for i in train_df.index:
  arg = train_df['argument'][i]
  label = train_df['label'][i]
  if label == 1:
    if arg[-1] != '.':
      modified_arg = arg + '. This means '
      train_df.at[i, 'argument'] = modified_arg
    else:
      modified_arg = arg + ' This means '
      train_df.at[i, 'argument'] = modified_arg
  else:
    if arg[-1] != '.':
      modified_arg = arg + '. This does not mean '
      train_df.at[i, 'argument'] = modified_arg
    else:
      modified_arg = arg + ' This does not mean '
      train_df.at[i, 'argument'] = modified_arg

# For Dev set

for i in dev_df.index:
  arg = dev_df['argument'][i]
  label = dev_df['label'][i]
  if label == 1:
    if arg[-1] != '.':
      modified_arg = arg + '. This means '
      dev_df.at[i, 'argument'] = modified_arg
    else:
      modified_arg = arg + ' This means '
      dev_df.at[i, 'argument'] = modified_arg
  else:
    if arg[-1] != '.':
      modified_arg = arg + '. This does not mean '
      dev_df.at[i, 'argument'] = modified_arg
    else:
      modified_arg = arg + ' This does not mean '
      dev_df.at[i, 'argument'] = modified_arg

# For Test set

for i in test_df.index:
  arg = test_df['argument'][i]
  label = test_df['label'][i]
  if label == 1:
    if arg[-1] != '.':
      modified_arg = arg + '. This means '
      test_df.at[i, 'argument'] = modified_arg
    else:
      modified_arg = arg + ' This means '
      test_df.at[i, 'argument'] = modified_arg
  else:
    if arg[-1] != '.':
      modified_arg = arg + '. This does not mean '
      test_df.at[i, 'argument'] = modified_arg
    else:
      modified_arg = arg + ' This does not mean '
      test_df.at[i, 'argument'] = modified_arg

This is Template 2

In [ ]:
# Template 2
# For Train set

for i in train_df.index:
  arg = train_df['argument'][i]
  if arg[-1] != '.':
    modified_arg = 'What are the keypoints for the following argument? ' + arg + '.'
    train_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = 'What are the keypoints for the following argument? ' + arg
    train_df.at[i, 'argument'] = modified_arg

# For Dev set

for i in dev_df.index:
  arg = dev_df['argument'][i]
  if arg[-1] != '.':
    modified_arg = 'What are the keypoints for the following argument? ' + arg + '.'
    dev_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = 'What are the keypoints for the following argument? ' + arg
    dev_df.at[i, 'argument'] = modified_arg


# For Test set

for i in test_df.index:
  arg = test_df['argument'][i]
  if arg[-1] != '.':
    modified_arg = 'What are the keypoints for the following argument? ' + arg + '.'
    test_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = 'What are the keypoints for the following argument? ' + arg
    test_df.at[i, 'argument'] = modified_arg

Check a sample whether or not everything is okay.

In [12]:
train_df.at[110, 'argument']

'human cloning allows experimentation which might lead to the development of a more robust strain of human therefore it should not be banned. This does not mean '

In [13]:
train_df = train_df_prev.filter(['argument', 'key_point'], axis=1)
dev_df = dev_df_prev.filter(['argument', 'key_point'], axis=1)
test_df = test_df_prev.filter(['argument', 'key_point'], axis=1)

Rename the columns for compatibility with Simple Transformers

In [14]:
# Renaming the columns
train_df.columns = ["input_text","target_text"]
dev_df.columns = ["input_text","target_text"]
test_df.columns = ["input_text","target_text"]

In [15]:
# Adding a prefix. Here we shall keep "paraphrase" as a prefix.
train_df["prefix"] = "paraphrase"
dev_df["prefix"] = "paraphrase"
test_df["prefix"] = "paraphrase"

In [16]:
train_data, dev_data = train_df, dev_df

Data pre-processing is complete. Let's fine-tune T5 now!

In [17]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "evaluate_during_training": False,
    "adam_epsilon": 1e-08,
    "eval_batch_size": 6,
    "fp_16": False,
    "gradient_accumulation_steps": 16,
    "learning_rate": 0.0003,
    "max_grad_norm": 1.0,
    "n_gpu": 1,
    "seed": 42,
    "train_batch_size": 6,
    "warmup_steps": 0,
    "weight_decay": 0.0,
}

We are using T5-small here. You can use T5-base or T5-large as well.

In [18]:
from simpletransformers.t5 import T5Model, T5Args
import sklearn

#model_args = T5Args()
#model_args.num_train_epochs = 4

model = T5Model("t5","t5-small", args=args)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Training starts from here.

In [19]:
import sklearn.metrics

model.train_model(train_data, eval_data=dev_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/17019 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/2837 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/2837 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/2837 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/2837 [00:00<?, ?it/s]

(708, 0.7739943952699544)

Don''t forget to zip the model files and then upload it to your Google Drive

In [20]:
!zip -r /content/t5_small_a2_tem_1_full.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/config.json (deflated 63%)
  adding: content/outputs/spiece.model (deflated 48%)
  adding: content/outputs/model_args.json (deflated 63%)
  adding: content/outputs/tokenizer_config.json (deflated 79%)
  adding: content/outputs/checkpoint-708-epoch-4/ (stored 0%)
  adding: content/outputs/checkpoint-708-epoch-4/config.json (deflated 63%)
  adding: content/outputs/checkpoint-708-epoch-4/spiece.model (deflated 48%)
  adding: content/outputs/checkpoint-708-epoch-4/model_args.json (deflated 63%)
  adding: content/outputs/checkpoint-708-epoch-4/tokenizer_config.json (deflated 79%)
  adding: content/outputs/checkpoint-708-epoch-4/pytorch_model.bin (deflated 7%)
  adding: content/outputs/checkpoint-708-epoch-4/training_args.bin (deflated 49%)
  adding: content/outputs/checkpoint-708-epoch-4/scheduler.pt (deflated 51%)
  adding: content/outputs/checkpoint-708-epoch-4/optimizer.pt (deflated 15%)
  adding: content/outputs/checkpoint

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Get the model from Google drive if you previously trained it.

In [ ]:
!unzip /content/gdrive/MyDrive/bart_large-model_tem_1.zip

Archive:  /content/gdrive/MyDrive/bart_large-model_tem_1.zip
   creating: content/outputs/
   creating: content/outputs/checkpoint-2000/
 extracting: content/outputs/checkpoint-2000/eval_results.txt  
   creating: content/outputs/checkpoint-3512-epoch-8/
 extracting: content/outputs/checkpoint-3512-epoch-8/eval_results.txt  
   creating: content/outputs/checkpoint-3951-epoch-9/
 extracting: content/outputs/checkpoint-3951-epoch-9/eval_results.txt  
   creating: content/outputs/checkpoint-1317-epoch-3/
 extracting: content/outputs/checkpoint-1317-epoch-3/eval_results.txt  
   creating: content/outputs/checkpoint-2195-epoch-5/
 extracting: content/outputs/checkpoint-2195-epoch-5/eval_results.txt  
   creating: content/outputs/checkpoint-4000/
 extracting: content/outputs/checkpoint-4000/eval_results.txt  
   creating: content/outputs/checkpoint-1756-epoch-4/
 extracting: content/outputs/checkpoint-1756-epoch-4/eval_results.txt  
   creating: content/outputs/checkpoint-4390-epoch-10/
 ext

Evaluate the model now. You can pass an input to the model manually and observe the output. Or you can pass an entire test set and get the list of outputs.

In [22]:
from simpletransformers.t5 import T5Model
import os
root_dir = os.getcwd()
trained_model_path = os.path.join(root_dir,"/content/outputs")
args = {
"overwrite_output_dir": True,
"max_seq_length": 256,
"max_length": 50,
"top_k": 50,
"top_p": 0.95,
"num_return_sequences": 1
}
trained_model = T5Model("t5",trained_model_path,args=args)

In [ ]:
# Predict an input

prefix = "paraphrase"
pred = trained_model.predict([f"{prefix}: Under austerity regimes, public services and other societal benefits are reduced. This means "])
print(pred)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Pensions increase awareness/prudence']


In [23]:
# Predict full test set

prefix = "paraphrase"
ref = []
to_predict = []
for i in train_df.index:
  input = prefix + ": " + train_df['input_text'][i]
  ref.append(train_df['target_text'][i])
  to_predict.append(input)
  #pred.append(trained_model.predict(f"{prefix}: {input}"))

predictions = trained_model.predict(to_predict)

Generating outputs:   0%|          | 0/2837 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/17019 [00:00<?, ?it/s]

In [24]:
train_df.head(10)

,input_text,target_text,prefix
0,a person created through cloning could potenti...,Cloning is not understood enough yet,paraphrase
1,a person created through cloning could potenti...,Cloning is unethical/anti-religious,paraphrase
2,a person created through cloning could potenti...,Cloning is unnatural,paraphrase
3,a person created through cloning could potenti...,Cloning is unsafe,paraphrase
4,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,paraphrase
5,because it's unethical and unnatural,Cloning is not understood enough yet,paraphrase
6,because it's unethical and unnatural,Cloning is unethical/anti-religious,paraphrase
7,because it's unethical and unnatural,Cloning is unnatural,paraphrase
8,because it's unethical and unnatural,Cloning is unsafe,paraphrase
9,"by advancing cloning, scientists will have as ...",Cloning can be used for organ replication,paraphrase


In [25]:
train_df_prev.head(10)

,Unnamed: 0,topic,argument,key_point,stance,label
0,10137,We should ban human cloning,a person created through cloning could potenti...,Cloning is not understood enough yet,1,0
1,10138,We should ban human cloning,a person created through cloning could potenti...,Cloning is unethical/anti-religious,1,0
2,10139,We should ban human cloning,a person created through cloning could potenti...,Cloning is unnatural,1,0
3,10140,We should ban human cloning,a person created through cloning could potenti...,Cloning is unsafe,1,1
4,10141,We should ban human cloning,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,1,0
5,10142,We should ban human cloning,because it's unethical and unnatural,Cloning is not understood enough yet,1,0
6,10143,We should ban human cloning,because it's unethical and unnatural,Cloning is unethical/anti-religious,1,1
7,10144,We should ban human cloning,because it's unethical and unnatural,Cloning is unnatural,1,1
8,10145,We should ban human cloning,because it's unethical and unnatural,Cloning is unsafe,1,0
9,10146,We should ban human cloning,"by advancing cloning, scientists will have as ...",Cloning can be used for organ replication,-1,0


Finally, look at the accuracy, f1-score, precision and recall

In [26]:
predictions

['Cloning is unnatural',
 'Cloning is unnatural',
 'Cloning can be used for organ replication',
 'Cloning is unethical/anti-religious',
 'Cloning is unethical/anti-religious',
 'Sex selection will create abortions/infanticide',
 'It is unethical/unhealthy for parents to intervene',
 'Ghost hunting is necessary for hunting/sports-shooting',
 'Cloning is unethical/anti-religious',
 'Cloning promotes science/research',
 'Cloning promotes science/research',
 'Cloning promotes science/research',
 'Cloning helps those who can’t otherwise have a child',
 'Cloning promotes science/research',
 'Cloning promotes health',
 'Cloning can be used for organ replication',
 'Cloning promotes science/research',
 'Cloning can be used for organ replication',
 'Cloning is unnatural',
 'Cloning is unnatural',
 'Cloning is not understood enough yet',
 'Cloning is unnatural',
 'Cloning is unnatural',
 'Cloning can be used to create more advanced humans',
 'Cloning is unsafe',
 'Cloning can be used for organ r

In [27]:
train_df_prev['intermediary_text'] = predictions

In [28]:
train_df_prev.head(10)

,Unnamed: 0,topic,argument,key_point,stance,label,intermediary_text
0,10137,We should ban human cloning,a person created through cloning could potenti...,Cloning is not understood enough yet,1,0,Cloning is unnatural
1,10138,We should ban human cloning,a person created through cloning could potenti...,Cloning is unethical/anti-religious,1,0,Cloning is unnatural
2,10139,We should ban human cloning,a person created through cloning could potenti...,Cloning is unnatural,1,0,Cloning can be used for organ replication
3,10140,We should ban human cloning,a person created through cloning could potenti...,Cloning is unsafe,1,1,Cloning is unethical/anti-religious
4,10141,We should ban human cloning,Allowing humans to be cloned is setting the wo...,Cloning is unethical/anti-religious,1,0,Cloning is unethical/anti-religious
5,10142,We should ban human cloning,because it's unethical and unnatural,Cloning is not understood enough yet,1,0,Sex selection will create abortions/infanticide
6,10143,We should ban human cloning,because it's unethical and unnatural,Cloning is unethical/anti-religious,1,1,It is unethical/unhealthy for parents to inter...
7,10144,We should ban human cloning,because it's unethical and unnatural,Cloning is unnatural,1,1,Ghost hunting is necessary for hunting/sports-...
8,10145,We should ban human cloning,because it's unethical and unnatural,Cloning is unsafe,1,0,Cloning is unethical/anti-religious
9,10146,We should ban human cloning,"by advancing cloning, scientists will have as ...",Cloning can be used for organ replication,-1,0,Cloning promotes science/research


In [29]:
train_df_prev.to_csv('/content/outputs/inter-text_from_T5-small_temp_1_full.csv')